In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive/', force_remount=True)
os.chdir("/content/gdrive/MyDrive/Usc/EE641/Project/")
print("current working directory is ", os.getcwd())
!pip install yacs

Mounted at /content/gdrive/
current working directory is  /content/gdrive/MyDrive/Usc/EE641/Project
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import seg_models as sm
import torch.optim.lr_scheduler as lr_scheduler
from utils import *
device = (torch.device('cuda') if torch.cuda.is_available()
                  else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [ ]:
path = "HDF5Data/infect_data.hdf5"
train_transforms = transforms.Compose([
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation((-10, 10))
                                ])

train_img_transforms = transforms.Compose([transforms.Normalize((0.5302),(0.2452))])
test_img_transforms = transforms.Compose([transforms.Normalize((0.5302),(0.2452))])

train_dataset = HDF5Dataset(path, 'train', transform=train_transforms, img_transform=train_img_transforms)
val_dataset = HDF5Dataset(path, 'val', img_transform=test_img_transforms)

batch_size = 128
num_works=2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_works)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_works)

In [ ]:
def train_infect_model(encoder_name, train_loader, val_loader,
                       n_epochs=50, stop=10, step_size=25):

    model = sm.Unet(
        encoder_name=encoder_name,
        encoder_weights='imagenet',
        in_channels=1,                  
        classes=1,
        decoder_channels = (256, 128, 64, 32, 16),
        activation=nn.Sigmoid                      
    ).to(device)

    # optimizer = optim.SGD(Unet_micronet_m0.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.RAdam(model.parameters(), lr=0.001)

    # scheduler1 = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    # scheduler2 = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100], gamma=0.1)
    scheduler3 = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
    # scheduler4 = None
    loss_fcn = DiceBCELoss()
    training_loop(
        model, optimizer,
        loss_fcn, train_loader,
        val_loader, scheduler = scheduler3,
        mask_name='infection mask',
        n_epochs=n_epochs, device=device, 
        save_path=f'trained_models/infect_Unet_{encoder_name}.pt',
        stop = stop)
    
    torch.cuda.empty_cache()

In [ ]:
name_list = [f'micronet_m{i}' for i in range(4)]
name_list.extend(['resnet18', 'mobilenet_v2'])
for encoder_name in name_list:
    print(f"{encoder_name}:")
    train_infect_model(encoder_name)
    torch.cuda.empty_cache()

micronet_m0:
Epoch 0, train loss: 1.520
Epoch 0, val loss: 1.352
Detect Improvement, Save Model
Epoch 1, train loss: 1.176
Epoch 1, val loss: 1.162
Detect Improvement, Save Model
Epoch 2, train loss: 0.949
Epoch 2, val loss: 0.908
Detect Improvement, Save Model
Epoch 3, train loss: 0.810
Epoch 3, val loss: 0.741
Detect Improvement, Save Model
Epoch 4, train loss: 0.694
Epoch 4, val loss: 0.644
Detect Improvement, Save Model
Epoch 5, train loss: 0.604
Epoch 5, val loss: 0.552
Detect Improvement, Save Model
Epoch 6, train loss: 0.523
Epoch 6, val loss: 0.495
Detect Improvement, Save Model
Epoch 7, train loss: 0.460
Epoch 7, val loss: 0.415
Detect Improvement, Save Model
Epoch 8, train loss: 0.407
Epoch 8, val loss: 0.368
Detect Improvement, Save Model
Epoch 9, train loss: 0.367
Epoch 9, val loss: 0.344
Detect Improvement, Save Model
Epoch 10, train loss: 0.347
Epoch 10, val loss: 0.344
Epoch 11, train loss: 0.330
Epoch 11, val loss: 0.311
Detect Improvement, Save Model
Epoch 12, train lo

Downloading: "http://www.svcl.ucsd.edu/projects/micronet/assets/micronet-m1.pth" to /root/.cache/torch/hub/checkpoints/micronet-m1.pth


  0%|          | 0.00/6.87M [00:00<?, ?B/s]

Epoch 0, train loss: 1.476
Epoch 0, val loss: 1.321
Detect Improvement, Save Model
Epoch 1, train loss: 1.116
Epoch 1, val loss: 1.202
Detect Improvement, Save Model
Epoch 2, train loss: 0.887
Epoch 2, val loss: 0.906
Detect Improvement, Save Model
Epoch 3, train loss: 0.756
Epoch 3, val loss: 0.712
Detect Improvement, Save Model
Epoch 4, train loss: 0.654
Epoch 4, val loss: 0.618
Detect Improvement, Save Model
Epoch 5, train loss: 0.572
Epoch 5, val loss: 0.541
Detect Improvement, Save Model
Epoch 6, train loss: 0.509
Epoch 6, val loss: 0.507
Detect Improvement, Save Model
Epoch 7, train loss: 0.454
Epoch 7, val loss: 0.432
Detect Improvement, Save Model
Epoch 8, train loss: 0.401
Epoch 8, val loss: 0.406
Detect Improvement, Save Model
Epoch 9, train loss: 0.381
Epoch 9, val loss: 0.381
Detect Improvement, Save Model
Epoch 10, train loss: 0.352
Epoch 10, val loss: 0.354
Detect Improvement, Save Model
Epoch 11, train loss: 0.331
Epoch 11, val loss: 0.362
Epoch 12, train loss: 0.313
Epo

Downloading: "http://www.svcl.ucsd.edu/projects/micronet/assets/micronet-m2.pth" to /root/.cache/torch/hub/checkpoints/micronet-m2.pth


...

  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch 0, train loss: 1.260
Epoch 0, val loss: 1.251
Detect Improvement, Save Model
Epoch 1, train loss: 0.978
Epoch 1, val loss: 0.865
Detect Improvement, Save Model
Epoch 2, train loss: 0.788
Epoch 2, val loss: 0.718
Detect Improvement, Save Model
Epoch 3, train loss: 0.640
Epoch 3, val loss: 0.636
Detect Improvement, Save Model
Epoch 4, train loss: 0.520
Epoch 4, val loss: 0.485
Detect Improvement, Save Model
Epoch 5, train loss: 0.434
Epoch 5, val loss: 0.424
Detect Improvement, Save Model
Epoch 6, train loss: 0.375
Epoch 6, val loss: 0.503
Epoch 7, train loss: 0.346
Epoch 7, val loss: 0.341
Detect Improvement, Save Model
Epoch 8, train loss: 0.305
Epoch 8, val loss: 0.338
Detect Improvement, Save Model
Epoch 9, train loss: 0.264
Epoch 9, val loss: 0.292
Detect Improvement, Save Model
Epoch 10, train loss: 0.275
Epoch 10, val loss: 0.324
Epoch 11, train loss: 0.244
Epoch 11, val loss: 0.364
Epoch 12, train loss: 0.260
Epoch 12, val loss: 0.358
Epoch 13, train loss: 0.235
Epoch 13, v

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

OutOfMemoryError: ignored

In [ ]:
batch_size = 64
num_works=2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_works)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_works)

In [ ]:
name_list = ['mobilenet_v2']
for encoder_name in name_list:
    print(f"{encoder_name}:")
    train_infect_model(encoder_name)
    torch.cuda.empty_cache()